In [ ]:
# Copyright 2020 Erik Härkönen. All rights reserved.
# This file is licensed to you under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License. You may obtain a copy
# of the License at http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software distributed under
# the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR REPRESENTATIONS
# OF ANY KIND, either express or implied. See the License for the specific language
# governing permissions and limitations under the License.

# Teaser: sequence of 3 interesting edits
%matplotlib inline
from notebook_init import *

rand = lambda : np.random.randint(np.iinfo(np.int32).max)
outdir = Path('out/figures/teaser')
makedirs(outdir, exist_ok=True)

In [ ]:
def setup_model(model_name, class_name, layer_name):
    global inst, model, lat_comp, lat_mean, lat_std

    use_w = 'StyleGAN' in model_name
    inst = get_instrumented_model(model_name, class_name, layer_name, device, use_w=use_w, inst=inst)
    model = inst.model

    pc_config = Config(components=80, n=1_000_000, batch_size=200,
        layer=layer_name, model=model_name, output_class=class_name, use_w=use_w)
    dump_name = get_or_compute(pc_config, inst)

    with np.load(dump_name) as data:
        lat_comp = data['lat_comp']
        lat_mean = data['lat_mean']
        lat_std = data['lat_stdev']

In [ ]:
def perform_edit(seeds, edit_sequence, save_images=False, crop=None):
    max_figs = 1000 if save_images else 10

    for seed in seeds[:max_figs]:
        w = model.sample_latent(1, seed=seed).cpu().numpy()
        w = [w]*model.get_max_latents()
        imgs = []
        
        # Starting point
        imgs.append(model.sample_np(w))
        
        # Perform edits in order
        for edit in edit_sequence:
            (idx, start, end, strength, invert) = configs[edit]
            
            # Find out coordinate of w along PC
            w_centered = w[0] - lat_mean
            w_coord = np.sum(w_centered.reshape(-1)*lat_comp[idx].reshape(-1)) / lat_std[idx]
            
            # Invert property if desired (e.g. flip rotation)
            # Otherwise reinforce existing
            if invert:
                sign = w_coord / np.abs(w_coord)
                target = -sign*strength # opposite side of mean
            else:
                target = strength
                
            delta = target - w_coord # offset vector
            
            for l in range(start, end):
                w[l] = w[l] + lat_comp[idx]*lat_std[idx]*delta
            imgs.append(model.sample_np(w))
        
        # Crop away black borders
        if crop:
            imgs = [img[crop[0]:-crop[1], crop[2]:-crop[3], :] for img in imgs]

        if save_images:
            # Save to disk
            for i, img in enumerate(imgs):
                Image.fromarray((img*255).astype(np.uint8)).save(outdir / f'teaser_{seed}_{i}.png')
        
        # Show in notebook
        strip = np.hstack(imgs)
        #strip = strip[::2, ::2, :] # 2x downscale for preview
        plt.figure(figsize=(30,5))
        plt.imshow(strip, interpolation='bilinear')
        plt.axis('off')
        plt.show()


In [ ]:
# (idx, edit_start, edit_end, strength, invert)
configs = {
    # StyleGAN2 cars W
    'Redness':          (22,  9, 11,   -8, False),
    'Horizontal flip':  ( 0,  0,  5,  2.0, True),
    'Add grass':        (41,  9, 11,  -18, False),
    'Blocky shape':     (16,  3,  6,   25, False),

    # BigGAN-512 irish_setter
    'Move right':       ( 0,  0, 15, -1.5, False),
    'Rotate':           ( 3,  0, 15,  -0.5, False),
    'Move back':        ( 4,  0, 15,  2.5, False),
    'Zoom in':          ( 6,  0, 15, -2.0, False),
    'Zoom out':         (12,  0, 15, -4.0, False),
    'Sharpen BG':       (13,  6,  9, 20.0, False),
    'Camera down':      (15,  1,  6, -4.0, False),
    'Light right':      (28,  7,  8,  30, False),
    'Pixelate':         (46, 10, 11,  -25, False),
    'Reeds':            (61,  4,  8,  -15, False),
    'Dry BG':           (65,  6,  8,  -30, False),
    'Grass length':     (69,  5,  8,   15, False),

    # StyleGAN2 ffhq
    'frizzy_hair':             (31,  2,  6,  20, False),
    'background_blur':         (49,  6,  9,  20, False),
    'bald':                    (21,  2,  5,  20, False),
    'big_smile':               (19,  4,  5,  20, False),
    'caricature_smile':        (26,  3,  8,  13, False),
    'scary_eyes':              (33,  6,  8,  20, False),
    'curly_hair':              (47,  3,  6,  20, False),
    'dark_bg_shiny_hair':      (13,  8,  9,  20, False),
    'dark_hair_and_light_pos': (14,  8,  9,  20, False),
    'dark_hair':               (16,  8,  9,  20, False),
    'disgusted':               (43,  6,  8, -30, False),
    'displeased':              (36,  4,  7,  20, False),
    'eye_openness':            (54,  7,  8,  20, False),
    'eye_wrinkles':            (28,  6,  8,  20, False),
    'eyebrow_thickness':       (37,  8,  9,  20, False),
    'face_roundness':          (37,  0,  5,  20, False),
    'fearful_eyes':            (54,  4, 10,  20, False),
    'hairline':                (21,  4,  5, -20, False),
    'happy_frizzy_hair':       (30,  0,  8,  20, False),
    'happy_elderly_lady':      (27,  4,  7,  20, False),
    'head_angle_up':           (11,  1,  4,  20, False),
    'huge_grin':               (28,  4,  6,  20, False),
    'in_awe':                  (23,  3,  6, -15, False),
    'wide_smile':              (23,  3,  6,  20, False),
    'large_jaw':               (22,  3,  6,  20, False),
    'light_lr':                (15,  8,  9,  10, False),
    'lipstick_and_age':        (34,  6, 11,  20, False),
    'lipstick':                (34, 10, 11,  20, False),
    'mascara_vs_beard':        (41,  6,  9,  20, False),
    'nose_length':             (51,  4,  5, -20, False),
    'elderly_woman':           (34,  6,  7,  20, False),
    'overexposed':             (27,  8, 18,  15, False),
    'screaming':               (35,  3,  7, -15, False),
    'short_face':              (32,  2,  6, -20, False),
    'show_front_teeth':        (59,  4,  5,  40, False),
    'smile':                   (46,  4,  5, -20, False),
    'straight_bowl_cut':       (20,  4,  5, -20, False),
    'sunlight_in_face':        (10,  8,  9,  10, False),
    'trimmed_beard':           (58,  7,  9,  20, False),
    'white_hair':              (57,  7, 10, -24, False),
    'wrinkles':                (20,  6,  7, -18, False),
    'boyishness':              (8,   2,  5,  20, False),
}

In [ ]:
# StyleGAN2 faces - emphasis on novel edits
setup_model('StyleGAN2', 'ffhq', 'style')
model.truncation = 0.7
model.use_w()

seeds = [6293435, 2105448342] # + [rand() for _ in range(1)]
print(seeds)
edits = ['wrinkles', 'white_hair', 'in_awe', 'overexposed']
perform_edit(seeds, edits, True)

In [ ]:
# StyleGAN2 cars
setup_model('StyleGAN2', 'car', 'style')
model.truncation = 0.6
model.use_w()

seeds = [440749230] # + [rand() for _ in range(10)]
edits = ['Redness', 'Horizontal flip', 'Add grass', 'Blocky shape']
perform_edit(seeds, edits, True, crop=[64, 64, 1, 1])

In [ ]:
# BigGAN-512 irish setter
setup_model('BigGAN-512', 'husky', 'generator.gen_z')
model.set_output_class('irish_setter')
model.truncation = 0.6

seeds = [489408325]# + [rand() for _ in range(10)]
edits = ['Rotate', 'Zoom out', 'Camera down', 'Reeds']
perform_edit(seeds, edits, True)